In [11]:
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
from numba import jit
from tqdm import tqdm
from itertools import combinations, permutations
import time

In [12]:
data_A = np.loadtxt('../data/TSPA.csv', delimiter=';').astype(np.int64)
data_B = np.loadtxt('../data/TSPB.csv', delimiter=';').astype(np.int64)
data_C = np.loadtxt('../data/TSPC.csv', delimiter=';').astype(np.int64)
data_D = np.loadtxt('../data/TSPD.csv', delimiter=';').astype(np.int64)

In [13]:
def create_cost_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2) + cost.reshape(1, -1)).round().astype(np.int64)

def create_dist_matrix(data):
    x = data[:, :1]
    y = data[:, 1:2]
    #cost = data[:, 2:3]
    return (((x - x.reshape(1, -1))**2 + (y - y.reshape(1, -1))**2) ** (1/2)).round().astype(np.int64)

In [14]:
cost_matrix_A = create_cost_matrix(data_A)

In [15]:
cost_matrix_A

array([[  84, 2032, 2098, ..., 4159, 3783, 1514],
       [1633,  483, 2398, ..., 3349, 2266,  817],
       [ 720, 1419, 1462, ..., 3640, 3149,  964],
       ...,
       [2782, 2371, 3641, ..., 1461, 2908, 2554],
       [2558, 1440, 3302, ..., 3060, 1309, 1773],
       [1234,  936, 2062, ..., 3651, 2718,  364]])

In [16]:
cost_matrix_B = create_cost_matrix(data_B)

In [17]:
cost_matrix_C = create_cost_matrix(data_C)

In [18]:
cost_matrix_D = create_cost_matrix(data_D)

In [19]:
dist_matrix_A = create_dist_matrix(data_A)
dist_matrix_B = create_dist_matrix(data_B)
dist_matrix_C = create_dist_matrix(data_C)
dist_matrix_D = create_dist_matrix(data_D)

In [20]:
def plot(data, solution):
    data_ordered = np.array([data[i] for i in solution])
    all_data = np.array([data[i] for i in range(200)])

    plt.figure(figsize=(10, 10), dpi=80)

    plt.scatter(data_ordered[:,0], data_ordered[:,1], s=data_ordered[:,2]/data_ordered[:,2].max()*200, c='b')
    plt.scatter(all_data[:,0], all_data[:,1], s=all_data[:,2]/all_data[:,2].max()*200, c='b')
    plt.plot(data_ordered[:,0], data_ordered[:,1], 'y-')
    plt.plot([data_ordered[0,0], data_ordered[-1,0]], [data_ordered[0,1], data_ordered[-1,1]], 'y-')
    plt.show()

In [21]:
def calculate_performance(cycle, cost_matrix):
    total_sum = 0
    for i in range(len(cycle)-1):
        total_sum += cost_matrix[cycle[i], cycle[i+1]]
    total_sum += cost_matrix[cycle[-1], cycle[0]]
    return total_sum

In [22]:
np.arange(0, 100)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [23]:
@jit()
def random_solution(cost_matrix, limit=100):
    random_solution_list = np.arange(0, 100)
    np.random.shuffle(random_solution_list)
    return random_solution_list[:limit]

solution = random_solution(cost_matrix_A, 100)
print(solution)

[15 94 26 28 61 71 33  3  4  5 55 48 60 91 68 53 34 10 93 21 80 72 79 47
 31 41 82 51  7  6 22 18 83  9 13 20 58 96 25 88 12 30  1 16 37 92 32 57
 38 66 74 27 89 52 19 99 90 23  0 98 85 67 49 44 11 62 70 75 43 59 50 78
 95 81 97 63 45 77 87 84 39 29 64  2 35 56 46 36 40 76 86 14 73 42 69 24
 65  8 17 54]


In [24]:
# %%timeit
# random_solution(cost_matrix_A)

In [25]:
@jit()
def greedy_cycle(cost_matrix, current_id, limit=100):
    all_ids = set(list(range(0,len(cost_matrix))))
    all_ids.remove(current_id)
    solution = [current_id]
    
    for _ in range(1):
        min_val = 99999
        min_id = -1
        for next_id in all_ids:
            if cost_matrix[current_id][next_id] < min_val:
                min_val = cost_matrix[current_id][next_id]
                min_id = next_id
        solution.append(min_id)
        all_ids.remove(min_id)
        current_id = min_id
    
    while len(solution) < limit:
        min_delta = 99999
        min_id = -1
        insert_id = -1
        for i in range(len(solution)-1):
            for next_id in all_ids:
                delta = cost_matrix[solution[i]][next_id] + cost_matrix[next_id][solution[i+1]] - cost_matrix[solution[i]][solution[i+1]]
                if delta < min_delta:
                    min_delta = delta
                    min_id = next_id
                    insert_id = i
        for next_id in all_ids:
            delta = cost_matrix[solution[-1]][next_id] + cost_matrix[next_id][solution[0]] - cost_matrix[solution[-1]][solution[0]]
            if delta < min_delta:
                min_delta = delta
                min_id = next_id
                insert_id = i
        solution.insert(insert_id+1, min_id)
        all_ids.remove(min_id)

    return np.array(solution)

In [26]:
calculate_performance(greedy_cycle(cost_matrix_A, 2), cost_matrix_A)

77328

In [27]:
jit(nopython=True)
def calculate_closest_neighbours(cost_matrix, n_neighbours=10):
    def extract_nodes(ll):
        return [x[1] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix + diag_matrix

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[i] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [28]:
jit(nopython=True)
def calculate_closest_neighbours_solution(cost_matrix, solution, n_neighbours=10):
    def extract_nodes(ll):
        return [solution[x[1]] for x in ll]
    
    diag_matrix = np.diag(range(len(cost_matrix))).astype(float)
    np.fill_diagonal(diag_matrix, np.inf)
    
    new_cost_matrix = cost_matrix[solution, :][:, solution] + diag_matrix[solution, :][:, solution]

    closest_neighbours = dict()
    for i in range(len(new_cost_matrix)):
        closest_neighbours[solution[i]] = extract_nodes(sorted(zip(new_cost_matrix[i], range(len(new_cost_matrix))), key=lambda x: x[0])[:n_neighbours])
    return closest_neighbours

In [29]:
calculate_closest_neighbours(cost_matrix_A)

{0: [19, 178, 149, 114, 4, 55, 164, 50, 76, 43],
 1: [177, 75, 189, 41, 199, 48, 152, 11, 119, 4],
 2: [4, 114, 77, 0, 43, 149, 75, 175, 167, 91],
 3: [178, 0, 164, 19, 128, 59, 132, 143, 96, 55],
 4: [114, 77, 0, 43, 149, 19, 121, 91, 75, 175],
 5: [135, 95, 98, 112, 169, 72, 6, 51, 167, 8],
 6: [98, 141, 66, 186, 72, 156, 135, 172, 79, 190],
 7: [74, 163, 195, 55, 62, 113, 117, 22, 53, 32],
 8: [95, 169, 80, 26, 135, 48, 119, 31, 189, 75],
 9: [167, 75, 189, 135, 177, 186, 4, 114, 101, 119],
 10: [132, 55, 74, 113, 163, 185, 128, 195, 96, 59],
 11: [48, 152, 75, 177, 189, 106, 119, 1, 160, 26],
 12: [98, 94, 95, 72, 31, 6, 135, 169, 73, 112],
 13: [26, 48, 8, 95, 119, 169, 106, 75, 189, 135],
 14: [111, 95, 80, 31, 8, 98, 94, 169, 72, 135],
 15: [117, 55, 195, 22, 74, 53, 62, 79, 163, 108],
 16: [48, 75, 152, 11, 189, 177, 119, 106, 130, 1],
 17: [75, 189, 177, 167, 4, 135, 114, 119, 199, 1],
 18: [55, 117, 195, 22, 74, 53, 163, 62, 113, 79],
 19: [178, 0, 149, 164, 114, 4, 43, 128, 

In [30]:
calculate_closest_neighbours_solution(cost_matrix_A, [1,2,3,4,5,6,7,8,9,10,11,12,13])

{1: [11, 4, 8, 6, 9, 2, 5, 12, 13, 7],
 2: [4, 1, 6, 11, 8, 9, 5, 12, 7, 10],
 3: [4, 10, 6, 1, 7, 2, 11, 8, 9, 5],
 4: [1, 2, 6, 11, 8, 9, 5, 12, 7, 3],
 5: [6, 8, 12, 4, 1, 11, 9, 13, 2, 7],
 6: [8, 12, 4, 5, 1, 11, 9, 7, 2, 13],
 7: [6, 4, 10, 1, 8, 3, 2, 5, 12, 11],
 8: [11, 6, 1, 12, 5, 4, 13, 9, 2, 7],
 9: [4, 1, 6, 8, 11, 5, 2, 12, 13, 7],
 10: [4, 7, 6, 3, 1, 2, 8, 11, 9, 5],
 11: [1, 8, 4, 6, 13, 5, 9, 12, 2, 7],
 12: [6, 8, 5, 4, 11, 1, 9, 13, 2, 7],
 13: [8, 11, 1, 6, 4, 5, 12, 9, 2, 7]}

In [31]:
pre = calculate_closest_neighbours_solution(cost_matrix_A, list(range(100)))

In [32]:
solution = list(range(15)) + list(range(100, 115))

In [33]:
# %%timeit
# calculate_closest_neighbours_solution(cost_matrix_A, solution)

In [34]:
def calculate_change_before(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_2-1]] -
            ( cost_matrix[enhanced_solution[position_1-1], enhanced_solution[position_1]] + cost_matrix[enhanced_solution[position_2-1], enhanced_solution[position_2]] )
            )

def calculate_change_after(enhanced_solution, cost_matrix, position_1, position_2):
    return (cost_matrix[enhanced_solution[position_1], enhanced_solution[position_2]] + cost_matrix[enhanced_solution[(position_1+1)%len(enhanced_solution)], enhanced_solution[(position_2+1)%len(enhanced_solution)]] -
            ( cost_matrix[enhanced_solution[position_1], enhanced_solution[(position_1+1)%len(enhanced_solution)]] + cost_matrix[enhanced_solution[position_2], enhanced_solution[(position_2+1)%len(enhanced_solution)]] )
            )

In [35]:
def calculate_change_inter(enhanced_solution, cost_matrix, position_1, node_closest):
    return (cost_matrix[enhanced_solution[position_1], node_closest] + cost_matrix[node_closest, enhanced_solution[(position_1+2)%len(enhanced_solution)]] -
           (cost_matrix[enhanced_solution[position_1], enhanced_solution[(position_1+1)%len(enhanced_solution)]] + cost_matrix[enhanced_solution[(position_1+1)%len(enhanced_solution)], enhanced_solution[(position_1+2)%len(enhanced_solution)]])
           )

In [36]:
import time

In [37]:
# @jit()
def enhance_solution(initial_solution, closest_neighbours, distance_matrix, cost_matrix):
    start_all = time.time()
    enhanced_solution = list(initial_solution)
    previous_solution = 0
    current_solution = calculate_performance(initial_solution, cost_matrix)
    
    total_time_rest = 0
    total_time_edges = 0
    total_time_inter = 0
    while True:
        best_type = None

        ## Edges
        previous_solution = current_solution
        best_delta_edges = 0                  # e.g. -10
        best_edges_positions = None     # e.g. [1, 10]
        best_type = None                # before or after

        best_delta_inter = 0
        best_positions_inter = None

        best_operation = None

        neighbours_positions = {node: enhanced_solution.index(node) for node in enhanced_solution}
        
        start_rest = time.time()
        for position in range(len(initial_solution)):
            for neighbour in closest_neighbours[enhanced_solution[position]]:
                ## Intra edges
                if neighbour in enhanced_solution:
                    start = time.time()
                    position_neighbour = neighbours_positions[neighbour]
                    
                    if abs(position - position_neighbour) == 1: continue
                        
                    
                    for swap_type in ('before', 'after'):
                        
                        if swap_type == 'before':
                            delta = calculate_change_before(enhanced_solution, distance_matrix, position_1=position, position_2=position_neighbour)
                            
                        elif swap_type == 'after':
                            delta = calculate_change_after(enhanced_solution, distance_matrix, position_1=position, position_2=position_neighbour)
                        
                        if delta < best_delta_edges:
                            best_delta_edges = delta
                            best_edges_positions = [position, position_neighbour]
                            best_type = swap_type
                    end = time.time()
                    total_time_edges += end - start
                ##Inter
                else:
                    start = time.time()
                    delta = calculate_change_inter(enhanced_solution, cost_matrix, position_1=position, node_closest=neighbour)
                    if delta < best_delta_inter:
                        best_delta_inter = delta
                        best_positions_inter = [position, neighbour]
                    end = time.time()
                    total_time_inter += end - start 

        end_rest = time.time()
        total_time_rest += end_rest - start_rest
           
        
        if any([best_delta_inter != 0, best_delta_edges != 0]):
            if best_delta_edges < best_delta_inter:
                best_operation = 'edges'
            else:
                best_operation = 'inter'
        elif all([best_delta_inter == 0, best_delta_edges == 0]):
            end_all = time.time()
            return enhanced_solution, total_time_edges, total_time_inter, end_all - start_all, total_time_rest

        #Edges
        if best_operation == 'edges':
            if best_delta_edges == 0:
                print("XD")
                return enhanced_solution, total_time_edges, total_time_inter
            else:
                if best_type == 'before':
                    if best_edges_positions[0] < best_edges_positions[1]:
                        enhanced_solution = enhanced_solution[:best_edges_positions[0]] + enhanced_solution[best_edges_positions[0]:best_edges_positions[1]][::-1] + enhanced_solution[best_edges_positions[1]:]
                    else:
                        enhanced_solution = enhanced_solution[best_edges_positions[1]:best_edges_positions[0]] + list(enhanced_solution[best_edges_positions[0]:] + enhanced_solution[:best_edges_positions[1]])[::-1]
                elif best_type == 'after':
                    if best_edges_positions[0] < best_edges_positions[1]:
                        enhanced_solution = enhanced_solution[:(best_edges_positions[0]+1)%len(enhanced_solution)] + enhanced_solution[(best_edges_positions[0]+1)%len(enhanced_solution):(best_edges_positions[1]+1)][::-1] + enhanced_solution[(best_edges_positions[1]+1):]
                    else:
                        enhanced_solution = enhanced_solution[best_edges_positions[1]+1:best_edges_positions[0]+1] + list(enhanced_solution[(best_edges_positions[0]+1):] + enhanced_solution[:best_edges_positions[1]+1])[::-1]
        #Inter
        else:
            enhanced_solution[(best_positions_inter[0]+1)%len(enhanced_solution)] = best_positions_inter[1]
            pass
        
                
        current_solution = calculate_performance(enhanced_solution, cost_matrix)
        assert current_solution < previous_solution

        
        


In [38]:
from tqdm import tqdm
import sys

In [39]:
def start_enhancement(starting_solution_type: str, cost_matrix, distance_matrix, n_neighbours=10):
    total_costs = []
    total_cost_random = 0
    total_time = 0
    total_time_edges = 0
    total_time_inter = 0
    total_time_rest = 0
    for i in tqdm(range(len(distance_matrix))):
        # print(i)
        if starting_solution_type == 'greedy':
            initial_solution = greedy_cycle(cost_matrix, i, limit=100)
        elif starting_solution_type == 'random':
            initial_solution = random_solution(cost_matrix, limit=100)
        else:
            return 'Bad type'
        initial_cost = calculate_performance(initial_solution, cost_matrix)
        total_cost_random += initial_cost

        closest_neighbours = calculate_closest_neighbours(distance_matrix, n_neighbours=n_neighbours)
        enhanced_solution, time_edges, time_inter, time_all, time_rest = enhance_solution(initial_solution, closest_neighbours, distance_matrix, cost_matrix)
        total_costs.append(calculate_performance(enhanced_solution, cost_matrix))
        total_time += time_all
        total_time_edges += time_edges
        total_time_inter += time_inter
        total_time_rest += time_rest

        ## Uncomment those lines and delete tqdm for it to work properly (useful for time improvement checking) 
        # sys.stdout.write("Total time (edges, inter, rest, everything) (%f, %f, %f, %f) Progress (%d / 200)  \r" % (total_time_edges, total_time_inter, total_time_rest, total_time, i+1) )
        # sys.stdout.flush()

    total_costs = np.array(total_costs)
    return (total_costs.mean(), total_costs.min(), total_costs.max()) #, total_cost_random / len(cost_matrix), total_time, total_time / len(cost_matrix)
        
        

        


In [40]:
def multiple_local_searches(cost_matrix, distance_matrix, n_neighbours=10, iterations=200, rounds=20, start='random'):

    closest_neighbours = calculate_closest_neighbours(distance_matrix, n_neighbours=n_neighbours)

    results_each_iteration = []
    start_time = time.time()
    for i in range(rounds):
        print(i)
        results_iteration = []
        for j in tqdm(range(iterations)):
            if start == 'random':
                initial_solution = random_solution(cost_matrix, limit=100)
            else:
                return 'Yikes'

            enhanced_solution, time_edges, time_inter, time_all, time_rest = enhance_solution(initial_solution, closest_neighbours, distance_matrix, cost_matrix)
            results_iteration.append(calculate_performance(enhanced_solution, cost_matrix))
        results_each_iteration.append(results_iteration)
    return results_each_iteration, (time.time() - start_time) / rounds
    

In [41]:
def permute(route, permutation_size):
    length_solution = len(route)
    start = random.randint(0, length_solution-permutation_size-1)
    end = start + permutation_size
    return list(route[:start]) + list(route[start:end][::-1]) + list(route[end:])

In [42]:
def iterated_local_search(cost_matrix, distance_matrix, n_neighbours=10, rounds=20, start='random', swap_size=3):
    closest_neighbours = calculate_closest_neighbours(distance_matrix, n_neighbours=n_neighbours)

    results_each_iteration = []
    for i in tqdm(range(rounds)):
        if start == 'random':
            initial_solution = random_solution(cost_matrix, limit=100)
        else:
            return 'Yikes'
        best_route = initial_solution
        best_performance = calculate_performance(initial_solution, cost_matrix)
        route = initial_solution
        start_time = time.time()
        while time.time() - start_time <  2.5*60:
            route = permute(route, swap_size)
            
            enhanced_solution, time_edges, time_inter, time_all, time_rest = enhance_solution(route, closest_neighbours, distance_matrix, cost_matrix)
            performace_new = calculate_performance(enhanced_solution, cost_matrix)
            

            if performace_new < best_performance:
                best_performance = performace_new
                route = enhanced_solution
                best_route = enhanced_solution

        results_each_iteration.append([best_performance, best_route])
    return results_each_iteration
                



## Check different permutation sizes

In [1]:
import pickle

In [34]:
swipe_sizes_C = dict()
for i in range(3, 11):
    print(i)
    results_C = iterated_local_search(cost_matrix=cost_matrix_C, distance_matrix=dist_matrix_C, rounds=20, swap_size=i)
    swipe_sizes_C['size_'+str(i)] = results_C

3


100%|██████████| 20/20 [50:02<00:00, 150.14s/it]


4


100%|██████████| 20/20 [50:02<00:00, 150.14s/it]


5


...

100%|██████████| 20/20 [50:03<00:00, 150.15s/it]


9


100%|██████████| 20/20 [50:03<00:00, 150.20s/it]


10


100%|██████████| 20/20 [50:04<00:00, 150.24s/it]


In [35]:
with open('dictionary_C.pickle', 'wb') as handle:
    pickle.dump(swipe_sizes_C, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [36]:
swipe_sizes_D = dict()
for i in range(3, 11):
    print(i)
    results_D = iterated_local_search(cost_matrix=cost_matrix_D, distance_matrix=dist_matrix_D, rounds=20, swap_size=i)
    swipe_sizes_D['size_'+str(i)] = results_D

3


100%|██████████| 20/20 [50:02<00:00, 150.12s/it]


4


100%|██████████| 20/20 [50:03<00:00, 150.15s/it]


5


...

100%|██████████| 20/20 [50:01<00:00, 150.10s/it]


9


100%|██████████| 20/20 [50:02<00:00, 150.15s/it]


10


100%|██████████| 20/20 [50:03<00:00, 150.16s/it]


In [37]:
with open('dataset_D.pickle', 'wb') as handle:
    pickle.dump(swipe_sizes_D, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [2]:
with open('dictionary_C.pickle', 'rb') as handle:
    dictionary_C = pickle.load(handle)

In [3]:
with open('dictionary_D.pickle', 'rb') as handle:
    dictionary_D = pickle.load(handle)

In [45]:
pd.DataFrame(dictionary_C['size_9']).iloc[:, 0]

0     47796
1     48634
2     47710
3     47616
4     49618
5     47708
6     47690
7     48213
8     47902
9     48366
10    48475
11    48214
12    47466
13    48109
14    48382
15    48393
16    48592
17    47367
18    47878
19    47880
Name: 0, dtype: int64

In [ ]:
pd.DataFrame(dictionary_C['size_9']).iloc[:, 0]

In [80]:
pd.DataFrame(dictionary_C).apply(lambda x: x.apply(lambda x: x[0])).mean()

size_3     48722.65
size_4     48615.60
size_5     48333.15
size_6     48298.45
size_7     48115.60
size_8     48421.30
size_9     48100.45
size_10    48360.90
dtype: float64

In [81]:
pd.DataFrame(dictionary_D).apply(lambda x: x.apply(lambda x: x[0])).mean()

size_3     44792.20
size_4     44442.55
size_5     44465.75
size_6     44365.90
size_7     44156.35
size_8     44329.60
size_9     44175.95
size_10    44216.55
dtype: float64

## Multiple local searches

In [82]:
results_c = multiple_local_searches(cost_matrix=cost_matrix_C, distance_matrix=dist_matrix_C, iterations=200, rounds=20)

0


100%|██████████| 200/200 [02:41<00:00,  1.24it/s]


1


100%|██████████| 200/200 [02:41<00:00,  1.24it/s]


2


...

100%|██████████| 200/200 [02:38<00:00,  1.26it/s]


18


100%|██████████| 200/200 [02:38<00:00,  1.26it/s]


19


100%|██████████| 200/200 [02:39<00:00,  1.25it/s]


In [83]:
with open('mls_dictionary_C.pickle', 'wb') as handle:
    pickle.dump(results_c, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [84]:
results_D = multiple_local_searches(cost_matrix=cost_matrix_D, distance_matrix=dist_matrix_D, iterations=200, rounds=20)

0


100%|██████████| 200/200 [02:44<00:00,  1.21it/s]


1


100%|██████████| 200/200 [02:44<00:00,  1.21it/s]


2


...

100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


18


100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


19


100%|██████████| 200/200 [02:42<00:00,  1.23it/s]


In [85]:
with open('mls_dictionary_D.pickle', 'wb') as handle:
    pickle.dump(results_D, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [126]:
with open('dictionary_C.pickle', 'rb') as handle:
    iterated_c = pickle.load(handle)

with open('dictionary_D.pickle', 'rb') as handle:
    iterated_d = pickle.load(handle)

with open('mls_dictionary_C.pickle', 'rb') as handle:
    mls_c = pickle.load(handle)

with open('mls_dictionary_D.pickle', 'rb') as handle:
    mls_d = pickle.load(handle)

In [127]:
(pd.DataFrame(mls_c[0]).min(axis=1) - pd.DataFrame(iterated_c['size_9']).iloc[:, 0]).mean()

1560.4

In [128]:
(pd.DataFrame(mls_d[0]).min(axis=1) - pd.DataFrame(iterated_d['size_9']).iloc[:, 0]).mean()

2407.45

In [129]:
pd.DataFrame(mls_d[0]).min(axis=1)

0     46157
1     46089
2     46887
3     47034
4     46635
5     46307
6     46786
7     46210
8     46593
9     46591
10    46685
11    46538
12    46578
13    46168
14    46377
15    47230
16    47148
17    46412
18    46906
19    46337
dtype: int64

In [130]:
pd.DataFrame(iterated_d['size_9']).iloc[:, 0].min()

43416

In [163]:
general_df = pd.DataFrame()

In [164]:
iterated_df_c = pd.DataFrame(iterated_c['size_9']).iloc[:, 0].agg(['min', 'mean', 'max']).to_frame().rename(columns={0: 'iterated_c'})
iterated_df_c.name = 'iterated_c'
general_df = pd.concat([general_df, iterated_df_c], axis=1)

In [165]:
general_df

,iterated_c
min,47367.00
mean,48100.45
max,49618.00


In [166]:
iterated_df_d = pd.DataFrame(iterated_d['size_9']).iloc[:, 0].agg(['min', 'mean', 'max']).to_frame().rename(columns={0: 'iterated_d'})
iterated_df_d.name = 'iterated_d'
general_df = pd.concat([general_df, iterated_df_d], axis=1)

In [167]:
general_df

,iterated_c,iterated_d
min,47367.00,43416.00
mean,48100.45,44175.95
max,49618.00,45011.00


In [168]:
mls_df_c = pd.DataFrame(mls_c[0]).min(axis=1).agg(['min', 'mean', 'max']).to_frame().rename(columns={0: 'mls_c'})
mls_df_c.name = 'mls_c'
general_df = pd.concat([general_df, mls_df_c], axis=1)

In [169]:
mls_df_d = pd.DataFrame(mls_d[0]).min(axis=1).agg(['min', 'mean', 'max']).to_frame().rename(columns={0: 'mls_d'})
mls_df_d.name = 'mls_d'
general_df = pd.concat([general_df, mls_df_d], axis=1)

In [180]:
general_df.assign(difference_iterated_vs_mls_C=lambda df_: df_.iterated_c - df_.mls_c,
                 difference_iterated_vs_mls_D=lambda df_: df_.iterated_d - df_.mls_d)

,iterated_c,iterated_d,mls_c,mls_d,difference_iterated_vs_mls_C,difference_iterated_vs_mls_D
min,47367.00,43416.00,48814.00,46089.0,-1447.0,-2673.00
mean,48100.45,44175.95,49660.85,46583.4,-1560.4,-2407.45
max,49618.00,45011.00,50212.00,47230.0,-594.0,-2219.00


In [190]:
pd.DataFrame(iterated_d['size_9']).iloc[:, 1].apply(lambda x: calculate_performance(x, cost_matrix_D)).mean()

44175.95

In [188]:
iterated_c['size_9'][0][1]

[31,
 73,
 89,
 94,
 12,
 72,
 98,
 156,
 6,
 66,
 190,
 112,
 51,
 196,
 135,
 99,
 101,
 167,
 186,
 127,
 88,
 153,
 161,
 76,
 145,
 55,
 195,
 22,
 18,
 53,
 117,
 15,
 108,
 171,
 21,
 194,
 79,
 87,
 141,
 144,
 102,
 154,
 81,
 180,
 32,
 62,
 155,
 163,
 74,
 113,
 61,
 71,
 20,
 64,
 185,
 25,
 181,
 36,
 132,
 128,
 37,
 59,
 96,
 27,
 147,
 143,
 159,
 164,
 178,
 19,
 69,
 0,
 149,
 50,
 121,
 91,
 114,
 4,
 77,
 43,
 192,
 199,
 137,
 41,
 177,
 1,
 75,
 189,
 109,
 119,
 130,
 152,
 11,
 48,
 92,
 26,
 8,
 110,
 169,
 95]

In [177]:
pd.DataFrame(iterated_d['size_9']).iloc[:, 1].apply(lambda x: calculate_performance(x, cost_matrix_D))

0     43713
1     43806
2     44265
3     43416
4     44378
5     44360
6     44937
7     43863
8     44136
9     44447
10    44308
11    44854
12    44327
13    45011
14    43641
15    43762
16    44090
17    44100
18    44255
19    43850
Name: 1, dtype: int64

In [98]:
pd.DataFrame(mls_c['size_9'][0])#.iloc[:, 0].agg(['min', 'mean', 'max']).to_frame()
#iterated_df_c.name = 'iterated_c'

TypeError: tuple indices must be integers or slices, not str

In [ ]:
iterated_df_c = pd.DataFrame(iterated_c['size_9']).iloc[:, 0].agg(['min', 'mean', 'max']).to_frame()
iterated_df_c.name = 'iterated_c'

In [95]:
iterated_df_c

'iterated_c'

In [102]:
pd.DataFrame(results_c[0]).min(axis=1)

0     50059
1     50144
2     49394
3     49644
4     49509
5     50172
6     48947
7     49690
8     49234
9     49574
10    49524
11    48814
12    49876
13    50212
14    49594
15    50031
16    49686
17    49783
18    49714
19    49616
dtype: int64

In [103]:
pd.DataFrame(results_D[0]).min(axis=1)

0     46157
1     46089
2     46887
3     47034
4     46635
5     46307
6     46786
7     46210
8     46593
9     46591
10    46685
11    46538
12    46578
13    46168
14    46377
15    47230
16    47148
17    46412
18    46906
19    46337
dtype: int64

In [100]:
pd.DataFrame(results_c[0]).iloc[:, :1].min()

0    50462
dtype: int64